<a href="https://colab.research.google.com/github/rochak94/AB-Testing/blob/main/AB_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

random.seed(42)

import warnings
warnings.filterwarnings("ignore")

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = "/content/drive/MyDrive/A-B testing/ab_test.csv"
data = pd.read_csv(path)

In [5]:
data.head(5)

,id,time,con_treat,page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


Changing the Columns name

In [6]:
data.columns=['user_id','timestamp','group','landing_page','converted']
data.head()

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [8]:
data.shape

(294478, 5)

Eleminating the repeated values

In [9]:
data.nunique()

user_id         290584
timestamp        35993
group                2
landing_page         2
converted            2
dtype: int64

Checking for NaN values

In [10]:
data.isna().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

creating new dataframes

In [18]:
treat_t=data[data['group']=='treatment'].shape[0]
new_page_t=data[data['landing_page']=='new_page'].shape[0]

diff=treat_t-new_page_t
pd.DataFrame({'No. of treatment people':[treat_t],'No. of new page':[new_page_t],'Diffrence':[diff]})

,No. of treatment people,No. of new page,Diffrence
0,147276,147239,37


In [19]:
data[(data['group']=='treatment')&(data['landing_page']=='old_page')]

,user_id,timestamp,group,landing_page,converted
308,857184,34:59.8,treatment,old_page,0
327,686623,26:40.7,treatment,old_page,0
357,856078,29:30.4,treatment,old_page,0
685,666385,11:54.8,treatment,old_page,0
713,748761,47:44.4,treatment,old_page,0
...,...,...,...,...,...
293773,688144,34:50.5,treatment,old_page,1
293817,876037,15:09.0,treatment,old_page,1
293917,738357,37:55.7,treatment,old_page,0
294014,813406,25:33.2,treatment,old_page,0


In [20]:
data_mismatch=data[(data["group"]=='treatment') & (data['landing_page']=='old_page')
                    | (data["group"]=='control')&(data['landing_page']=='new_page')]
data_mismatch

,user_id,timestamp,group,landing_page,converted
22,767017,58:15.0,control,new_page,0
240,733976,11:16.4,control,new_page,0
308,857184,34:59.8,treatment,old_page,0
327,686623,26:40.7,treatment,old_page,0
357,856078,29:30.4,treatment,old_page,0
...,...,...,...,...,...
294014,813406,25:33.2,treatment,old_page,0
294200,928506,32:10.5,control,new_page,0
294252,892498,11:10.5,treatment,old_page,0
294253,886135,49:20.5,control,new_page,0


In [23]:
print('No. of mismatched rows:',data_mismatch.shape[0])
print('percentage of mismatched rows:', round(data_mismatch.shape[0]/data.shape[0]*100,2),'%.')

No. of mismatched rows: 3893
percentage of mismatched rows: 1.32 %.


In [24]:
data_correct=data[(data["group"]=='treatment') & (data['landing_page']=='old_page')
                    | (data["group"]=='control')&(data['landing_page']=='new_page')]

In [25]:
data_correct.head()

,user_id,timestamp,group,landing_page,converted
22,767017,58:15.0,control,new_page,0
240,733976,11:16.4,control,new_page,0
308,857184,34:59.8,treatment,old_page,0
327,686623,26:40.7,treatment,old_page,0
357,856078,29:30.4,treatment,old_page,0


In [27]:
data_correct.shape

(3893, 5)

In [28]:
data_correct=data_correct.drop_duplicates("user_id")

In [29]:
print('Probablity of conversion regardless of the page they recieve', data_correct['converted'].mean()*100,'%.')

Probablity of conversion regardless of the page they recieve 12.432571281787824 %.


In [32]:
data_correct.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3893 entries, 22 to 294331
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       3893 non-null   int64 
 1   timestamp     3893 non-null   object
 2   group         3893 non-null   object
 3   landing_page  3893 non-null   object
 4   converted     3893 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 182.5+ KB


In [33]:
data_correct['user_id']=data_correct['user_id'].astype('str')
data_correct.groupby('group').mean()*100

,converted
group,
control,12.136929
treatment,12.722646


In [34]:
(data_correct['landing_page'].value_counts()/data_correct.shape[0])*100

old_page    50.475212
new_page    49.524788
Name: landing_page, dtype: float64

probablity of a customer receiving new page is 50%

On the basis of probablity

1) The probablity of an individual receiving a new page is 50%

2)Probablity of an individual converting regardless of the page they recieved is 11.96%

3)Probablity of an indidual in control group is 12.04%

4) probality of an individual in treatment group is 11.88%



AB testing begins

>H0:There is no significant difference between the conversion rate for old page and new page.

>H1:There is significant difference between the conversion rate for old page and new page





In [35]:
converted_old=data_correct[(data_correct['landing_page']=='old_page') & (data_correct['converted']==1)]['user_id'].nunique()

converted_new=data_correct[(data_correct['landing_page']=='new_page') & (data_correct['converted']==1)]['user_id'].nunique()

print(converted_old)
print(converted_new)

250
234


In [37]:
t_old=data_correct[data_correct['landing_page']=='old_page']['user_id'].nunique()
t_new=data_correct[data_correct['landing_page']=='new_page']['user_id'].nunique()

In [38]:
print(t_old)
print(t_new)

1965
1928


In [41]:
import statsmodels.api as sm

z_score,p_value=sm.stats.proportions_ztest(np.array([converted_old,converted_new]),np.array([t_old,t_new]),alternative='smaller')

In [42]:
z_score,p_value

(0.553768860044138, 0.7101314788723916)